In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [7]:
#Getting original data as per warning
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
#data changed to 'boston'
boston = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]]) 
target = raw_df.values[1::2, 2]

In [62]:
#Create a pandas dataframe
data = pd.DataFrame(data=boston_dataset.data,columns=boston_dataset.feature_names)
features = data.drop(['INDUS','AGE'], axis = 1)

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns = ['PRICE'])
features.head()

,CRIM,ZN,CHAS,NOX,RM,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,0.0,0.538,6.575,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,0.0,0.469,6.421,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,0.0,0.469,7.185,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,0.0,0.458,6.998,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,0.0,0.458,7.147,6.0622,3.0,222.0,18.7,396.90,5.33


In [68]:
features.loc[0][2]

0.0

In [24]:
#property_stats = np.ndarray(shape=(1,11))
#property_stats[0][0] = 0.2
property_stats = features.mean().values.reshape(1,11)
property_stats

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [80]:
regr = LinearRegression().fit(features,target)
fitted_vals = regr.predict(features)

#Calculate the MSE and RMSE using sklearn
MSE = mean_squared_error(target,fitted_vals)
RMSE = np.sqrt(MSE)
RMSE

0.18751213519713034

In [89]:
log_estimate = regr.predict (property_stats)
log_estimate

/Users/mrsuperb/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([[-3.98648785]])

In [69]:
def get_log_prices(nr_rooms,student_per_classroom,next_to_river = False,high_confidence=True):
    #Configure property
    property_stats[0][4] = nr_rooms
    property_stats[0][8] = student_per_classroom
    
    if next_to_river:
        property_stats[0][2] = 1
    else:
        property_stats[0][2] = 0
        
     #Make Preditions
    log_estimate = regr.predict (property_stats)[0][0]
        
    #Calc Range
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
    
   
    
    return log_estimate,upper_bound,lower_bound,interval


In [70]:
get_log_prices(3,20,next_to_river=True,high_confidence=False)

/Users/mrsuperb/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


(2.840847272701882, 3.0283594078990124, 2.653335137504752, 68)

In [71]:
np.median(boston_dataset.target)

21.2

In [59]:
zillow_median_price = 583.3
scale_factor = zillow_median_price / np.median(boston_dataset.target)
scale_factor

27.514150943396224

In [86]:
#Challenge

def get_normal_prices(nr_rooms,student_per_classroom,next_to_river = False,high_confidence=True):
    #Configure property
    property_stats[0][4] = nr_rooms
    property_stats[0][8] = student_per_classroom
    
    if next_to_river:
        property_stats[0][2] = 1
    else:
        property_stats[0][2] = 0
        
     #Make Preditions
    log_estimate = regr.predict (property_stats)[0][0]
    normal_estimate = np.e**(log_estimate)
    dollar_est = normal_estimate * 1000 * scale_factor
    rounded_est = np.around(dollar_est,-3)
    #Calc Range
    if high_confidence:
        upper_bound = rounded_est + 2*RMSE
        lower_bound = rounded_est - 2*RMSE
        interval = 95
    else:
        upper_bound = rounded_est + RMSE
        lower_bound = rounded_est - RMSE
        interval = 68
    
   
    
    return rounded_est,upper_bound,lower_bound,interval

In [90]:
get_normal_prices(2,30,next_to_river=True,high_confidence=False)

/Users/mrsuperb/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


(296000.0, 296000.1875121352, 295999.8124878648, 68)